In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm  

In [ ]:
IMG_SIZE = 224  # Image size required by MobileNetV2
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001
NUM_CLASSES = 7  # FER-2013 has 7 emotions (angry, disgust, fear, happy, sad, surprise, neutral)


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize image
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(30),  # Random rotation up to 30 degrees
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize to MobileNetV2 requirements
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
train_data = datasets.ImageFolder("path_to_training_data", transform=train_transform)
val_data = datasets.ImageFolder("path_to_validation_data", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
for epoch in range(EPOCHS):
    
    # Training Phase
    model.train()
    train_loss, train_correct = 0, 0
    for inputs, labels in tqdm(train_loader):

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track training metrics
        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()
    
    train_accuracy = train_correct / len(train_data)
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    
    # Validation Phase
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Track validation metrics
            val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).sum().item()
    
    val_accuracy = val_correct / len(val_data)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
torch.save(model.state_dict(), "emotion_model.pth")